In [1]:
#First we will download the pretrained network in *.mat format. The mat format is a matlab onject, and the scipy package
#in Python has a method that can read it. The link to download the mat object is here. We save this model in the same
#folder our Python script is for reference

#http://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat

In [2]:
import os
import scipy.misc
from scipy import io
import numpy as np
import tensorflow as tf

In [3]:
#Then we can start a graph session and declare the locations of or two images: the original image and the style image.
sess = tf.Session()
original_image_file = 'temp/original.jpg'
style_image_file = 'temp/style.jpg'

In [30]:
#We will set some parameters for our model: the location of the mat file, weights, the learning rate, number of
#generations, and how frequently we should output the intermediate image. For the weights, it helps to highly weight the 
#style image over the original image. These hyperparameters should be tuned for changes in the desired result. 
vgg_path = 'imagenet-vgg-verydeep-19.mat'
original_image_weight = 5.0
style_image_weight = 200.0
regularization_weight = 50.0
learning_rate = 0.1
generations = 10000
output_generations = 500

In [5]:
#Now we will load the two images with scipy and change the style image to fit the original image dimensions
original_image = scipy.misc.imread(original_image_file)
style_image = scipy.misc.imread(style_image_file)

#Get shape of target and make the style image the same
target_shape = original_image.shape
style_image = scipy.misc.imresize(style_image, target_shape[1] / style_image.shape[1])

/home/huzeyfekiran/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/home/huzeyfekiran/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/huzeyfekiran/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  import sys


In [6]:
#From the paper, we can define the layers in order of how they appeared. We will use the author's naming convention
vgg_layers = ['conv1_1', 'relu1_1',
              'conv1_2', 'relu1_2', 'pool1',
              'conv2_1', 'relu2_1',
              'conv2_2', 'relu2_2', 'pool2',
              'conv3_1', 'relu3_1',
              'conv3_2', 'relu3_2',
              'conv3_3', 'relu3_3',
              'conv3_4', 'relu3_4', 'pool3',
              'conv4_1', 'relu4_1',
              'conv4_2', 'relu4_2',
              'conv4_3', 'relu4_3',
              'conv4_4', 'relu4_4', 'pool4',
              'conv5_1', 'relu5_1',
              'conv5_2', 'relu5_2',
              'conv5_3', 'relu5_3',
              'conv5_4', 'relu5_4']

In [13]:
#Now we will define a function that will extract the parameters from the mat file:
def extract_net_info(path_to_params):
    vgg_data = scipy.io.loadmat(path_to_params)
    normalization_matrix = vgg_data['normalization'][0][0][0]
    mat_mean = np.mean(normalization_matrix, axis=(0,1))
    network_weights = vgg_data['layers'][0]
    return(mat_mean, network_weights)

In [14]:
#From the loaded weights and the layer definitions, we can recreate the network in TensorFlow with the following function.
#We will loop through each layer and assign the corresponding function with appropriate weights and biases, where
#applicable
def vgg_network(network_weights, init_image):
    network = {}
    image = init_image

    for i, layer in enumerate(vgg_layers):
        if layer[0] == 'c':
            weights, bias = network_weights[i][0][0][0][0]
            weights = np.transpose(weights, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            conv_layer = tf.nn.conv2d(image, tf.constant(weights), (1, 1, 1, 1), 'SAME')
            image = tf.nn.bias_add(conv_layer, bias)
        elif layer[0] == 'r':
            image = tf.nn.relu(image)
        else:
            image = tf.nn.max_pool(image, (1, 2, 2, 1), (1, 2, 2, 1), 'SAME')
        network[layer] = image
    return(network)

In [15]:
#The paper recommends a few strategies of assigning intermediate layers to the original and style images. While we should
#keep relu4_2 for the original image, we can try different combinations of the other reluX_1 layer outputs for the 
#style image
original_layer = 'relu4_2'
style_layers = ['relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'relu5_1']

In [16]:
#Next, we will run the above functino to get the weights and mean. We will also change the image shapes to have four
#dimensions by adding a dimension of size one to the beginning. TensorFlow's image operations act on four dimensions,
#so we must add the batch-size dimension:
normalization_mean, network_weights = extract_net_info(vgg_path)

shape = (1,) + original_image.shape
style_shape = (1,) + style_image.shape
original_features = {}
style_features = {}

In [17]:
#Next, we declare the image plaaceholder and create the network with that placeholder:
image = tf.placeholder('float',shape=shape)
vgg_net = vgg_network(network_weights,image)

In [18]:
#We now normalize the original image matrix and run it through the network 
original_minus_mean = original_image - normalization_mean
original_norm = np.array([original_minus_mean])
original_features[original_layer] = sess.run(vgg_net[original_layer], feed_dict={image: original_norm})

In [19]:
#We repeat the same procedure with each of the style layers that we chose before
image = tf.placeholder('float', shape=style_shape)
vgg_net = vgg_network(network_weights, image)
style_minus_mean = style_image - normalization_mean
style_norm = np.array([style_minus_mean])

for layer in style_layers:
    layer_output = sess.run(vgg_net[layer], feed_dict={image: style_norm})
    layer_output = np.reshape(layer_output, (-1, layer_output.shape[3]))
    style_gram_matrix = np.matmul(layer_output.T, layer_output) / layer_output.size
    style_features[layer] = style_gram_matrix

In [21]:
#In order to create the combined image, we will start with random noise and run it through the network,
initial = tf.random_normal(shape) * 0.05
image = tf.Variable(initial)
vgg_net = vgg_network(network_weights,image)

In [22]:
# Loss
original_loss = original_image_weight * (2 * tf.nn.l2_loss(vgg_net[original_layer] - original_features[original_layer]) /
                original_features[original_layer].size)

# Loss from Style Image
style_loss = 0
style_losses = []
for style_layer in style_layers:
    layer = vgg_net[style_layer]
    feats, height, width, channels = [x.value for x in layer.get_shape()]
    size = height * width * channels
    features = tf.reshape(layer, (-1, channels))
    style_gram_matrix = tf.matmul(tf.transpose(features), features) / size
    style_expected = style_features[style_layer]
    #style_temp_loss = sess.run(2 * tf.nn.l2_loss(style_gram_matrix - style_expected) / style_expected.size)
    #print('Layer: {}, Loss: {}'.format(style_layer, style_temp_loss))
    style_losses.append(2 * tf.nn.l2_loss(style_gram_matrix - style_expected) / style_expected.size)
style_loss += style_image_weight * tf.reduce_sum(style_losses)

# To Smooth the resuts, we add in total variation loss       
total_var_x = sess.run(tf.reduce_prod(image[:,1:,:,:].get_shape()))
total_var_y = sess.run(tf.reduce_prod(image[:,:,1:,:].get_shape()))
first_term = regularization_weight * 2
second_term_numerator = tf.nn.l2_loss(image[:,1:,:,:] - image[:,:shape[1]-1,:,:])
second_term = second_term_numerator / total_var_y
third_term = (tf.nn.l2_loss(image[:,:,1:,:] - image[:,:,:shape[2]-1,:]) / total_var_x)
total_variation_loss = first_term * (second_term + third_term)

# Combined Loss
loss = original_loss + style_loss + total_variation_loss

In [24]:
#We next declare our optimizer and training step and initalize all the variables in the model.
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_step = optimizer.minimize(loss)
sess.run(tf.global_variables_initializer())

In [ ]:
#We now loop through our training generations and print a status update every so often and save the temporary image. We
#will save the temporary image because it is hard to determine how far to run this algorithm as it can vary, depending on
#the images chose. It is best to err on the larger generations size, and stop when a temporary image appears to be a good
#stopping point
for i in range(generations):
    sess.run(train_step)
    #Print update and save temporary output
    print("Train Step # {}".format(i+1))
    if(i+1)%output_generations==0:
        print('Generation {} out of {}'.format(i+1,generations))
        image_eval = sess.run(image)
        best_image_add_mean = image_eval.reshape(shape[1:]) + normalization_mean
        output_file = 'temp_output_{}.jpg'.format(i)
        scipy.misc.imsave(output_file,best_image_add_mean)

Train Step # 1
Train Step # 2
Train Step # 3
Train Step # 4
Train Step # 5
Train Step # 6
Train Step # 7
Train Step # 8
Train Step # 9
Train Step # 10
Train Step # 11
Train Step # 12
Train Step # 13
Train Step # 14
Train Step # 15
Train Step # 16
Train Step # 17
Train Step # 18
Train Step # 19
Train Step # 20
Train Step # 21
Train Step # 22
Train Step # 23
Train Step # 24
Train Step # 25
Train Step # 26
Train Step # 27
Train Step # 28
Train Step # 29
Train Step # 30
Train Step # 31
Train Step # 32
Train Step # 33
Train Step # 34
Train Step # 35
Train Step # 36
Train Step # 37
Train Step # 38
Train Step # 39
Train Step # 40
Train Step # 41
Train Step # 42
Train Step # 43
Train Step # 44
Train Step # 45
Train Step # 46
Train Step # 47
Train Step # 48
Train Step # 49
Train Step # 50
Train Step # 51
Train Step # 52
Train Step # 53
Train Step # 54
Train Step # 55
Train Step # 56
Train Step # 57
Train Step # 58
Train Step # 59
Train Step # 60
Train Step # 61
Train Step # 62
Train Step # 63
T

Train Step # 490
Train Step # 491
Train Step # 492
Train Step # 493
Train Step # 494
Train Step # 495
Train Step # 496
Train Step # 497
Train Step # 498
Train Step # 499
Train Step # 500
Generation 500 out of 10000


/home/huzeyfekiran/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


Train Step # 501
Train Step # 502
Train Step # 503
Train Step # 504
Train Step # 505
Train Step # 506
Train Step # 507
Train Step # 508
Train Step # 509
Train Step # 510
Train Step # 511
Train Step # 512
Train Step # 513
Train Step # 514
Train Step # 515
Train Step # 516
Train Step # 517
Train Step # 518
Train Step # 519
Train Step # 520
Train Step # 521
Train Step # 522
Train Step # 523
Train Step # 524
Train Step # 525
Train Step # 526
Train Step # 527
Train Step # 528
Train Step # 529
Train Step # 530
Train Step # 531
Train Step # 532
Train Step # 533
Train Step # 534
Train Step # 535
Train Step # 536
Train Step # 537
Train Step # 538
Train Step # 539
Train Step # 540
Train Step # 541
Train Step # 542
Train Step # 543
Train Step # 544
Train Step # 545
Train Step # 546
Train Step # 547
Train Step # 548
Train Step # 549
Train Step # 550
Train Step # 551
Train Step # 552
Train Step # 553
Train Step # 554
Train Step # 555
Train Step # 556
Train Step # 557
Train Step # 558
Train Step # 5

Train Step # 983
Train Step # 984
Train Step # 985
Train Step # 986
Train Step # 987
Train Step # 988
Train Step # 989
Train Step # 990
Train Step # 991
Train Step # 992
Train Step # 993
Train Step # 994
Train Step # 995
Train Step # 996
Train Step # 997
Train Step # 998
Train Step # 999
Train Step # 1000
Generation 1000 out of 10000
Train Step # 1001
Train Step # 1002
Train Step # 1003
Train Step # 1004
Train Step # 1005
Train Step # 1006
Train Step # 1007
Train Step # 1008
Train Step # 1009
Train Step # 1010
Train Step # 1011
Train Step # 1012
Train Step # 1013
Train Step # 1014
Train Step # 1015
Train Step # 1016
Train Step # 1017
Train Step # 1018
Train Step # 1019
Train Step # 1020
Train Step # 1021
Train Step # 1022
Train Step # 1023
Train Step # 1024
Train Step # 1025
Train Step # 1026
Train Step # 1027
Train Step # 1028
Train Step # 1029
Train Step # 1030
Train Step # 1031
Train Step # 1032
Train Step # 1033
Train Step # 1034
Train Step # 1035
Train Step # 1036
Train Step # 103

...

Train Step # 6888
Train Step # 6889
Train Step # 6890
Train Step # 6891
Train Step # 6892
Train Step # 6893
Train Step # 6894
Train Step # 6895
Train Step # 6896
Train Step # 6897
Train Step # 6898
Train Step # 6899
Train Step # 6900
Train Step # 6901
Train Step # 6902
Train Step # 6903
Train Step # 6904
Train Step # 6905
Train Step # 6906
Train Step # 6907
Train Step # 6908
Train Step # 6909
Train Step # 6910
Train Step # 6911
Train Step # 6912
Train Step # 6913
Train Step # 6914
Train Step # 6915
Train Step # 6916
Train Step # 6917
Train Step # 6918
Train Step # 6919
Train Step # 6920
Train Step # 6921
Train Step # 6922
Train Step # 6923
Train Step # 6924
Train Step # 6925
Train Step # 6926
Train Step # 6927
Train Step # 6928
Train Step # 6929
Train Step # 6930
Train Step # 6931
Train Step # 6932
Train Step # 6933
Train Step # 6934
Train Step # 6935
Train Step # 6936
Train Step # 6937
Train Step # 6938
Train Step # 6939
Train Step # 6940
Train Step # 6941
Train Step # 6942
Train Step

Train Step # 7342
Train Step # 7343
Train Step # 7344
Train Step # 7345
Train Step # 7346
Train Step # 7347
Train Step # 7348
Train Step # 7349
Train Step # 7350
Train Step # 7351
Train Step # 7352
Train Step # 7353
Train Step # 7354
Train Step # 7355
Train Step # 7356
Train Step # 7357
Train Step # 7358
Train Step # 7359
Train Step # 7360
Train Step # 7361
Train Step # 7362
Train Step # 7363
Train Step # 7364
Train Step # 7365
Train Step # 7366
Train Step # 7367
Train Step # 7368
Train Step # 7369
Train Step # 7370
Train Step # 7371
Train Step # 7372
Train Step # 7373
Train Step # 7374
Train Step # 7375
Train Step # 7376
Train Step # 7377
Train Step # 7378
Train Step # 7379
Train Step # 7380
Train Step # 7381
Train Step # 7382
Train Step # 7383
Train Step # 7384
Train Step # 7385
Train Step # 7386
Train Step # 7387
Train Step # 7388
Train Step # 7389
Train Step # 7390
Train Step # 7391
Train Step # 7392
Train Step # 7393
Train Step # 7394
Train Step # 7395
Train Step # 7396
Train Step

Train Step # 7796
Train Step # 7797
Train Step # 7798
Train Step # 7799
Train Step # 7800
Train Step # 7801
Train Step # 7802
Train Step # 7803
Train Step # 7804
Train Step # 7805
Train Step # 7806
Train Step # 7807
Train Step # 7808
Train Step # 7809
Train Step # 7810
Train Step # 7811
Train Step # 7812
Train Step # 7813
Train Step # 7814
Train Step # 7815
Train Step # 7816
Train Step # 7817
Train Step # 7818
Train Step # 7819
Train Step # 7820
Train Step # 7821
Train Step # 7822
Train Step # 7823
Train Step # 7824
Train Step # 7825
Train Step # 7826
Train Step # 7827
Train Step # 7828
Train Step # 7829
Train Step # 7830
Train Step # 7831
Train Step # 7832
Train Step # 7833
Train Step # 7834
Train Step # 7835
Train Step # 7836
Train Step # 7837
Train Step # 7838
Train Step # 7839
Train Step # 7840
Train Step # 7841
Train Step # 7842
Train Step # 7843
Train Step # 7844
Train Step # 7845
Train Step # 7846
Train Step # 7847
Train Step # 7848
Train Step # 7849
Train Step # 7850
Train Step

Train Step # 8250
Train Step # 8251
Train Step # 8252
Train Step # 8253
Train Step # 8254
Train Step # 8255
Train Step # 8256
Train Step # 8257
Train Step # 8258
Train Step # 8259
Train Step # 8260
Train Step # 8261
Train Step # 8262
Train Step # 8263
Train Step # 8264
Train Step # 8265
Train Step # 8266
Train Step # 8267
Train Step # 8268
Train Step # 8269
Train Step # 8270
Train Step # 8271
Train Step # 8272
Train Step # 8273
Train Step # 8274
Train Step # 8275
Train Step # 8276
Train Step # 8277
Train Step # 8278
Train Step # 8279
Train Step # 8280
Train Step # 8281
Train Step # 8282
Train Step # 8283
Train Step # 8284
Train Step # 8285
Train Step # 8286
Train Step # 8287
Train Step # 8288
Train Step # 8289
Train Step # 8290
Train Step # 8291
Train Step # 8292
Train Step # 8293
Train Step # 8294
Train Step # 8295
Train Step # 8296
Train Step # 8297
Train Step # 8298
Train Step # 8299
Train Step # 8300
Train Step # 8301
Train Step # 8302
Train Step # 8303
Train Step # 8304
Train Step

Train Step # 8704
Train Step # 8705
Train Step # 8706
Train Step # 8707
Train Step # 8708
Train Step # 8709
Train Step # 8710
Train Step # 8711
Train Step # 8712
Train Step # 8713
Train Step # 8714
Train Step # 8715
Train Step # 8716
Train Step # 8717
Train Step # 8718
Train Step # 8719
Train Step # 8720
Train Step # 8721
Train Step # 8722
Train Step # 8723
Train Step # 8724
Train Step # 8725
Train Step # 8726
Train Step # 8727
Train Step # 8728
Train Step # 8729
Train Step # 8730
Train Step # 8731
Train Step # 8732
Train Step # 8733
Train Step # 8734
Train Step # 8735
Train Step # 8736
Train Step # 8737
Train Step # 8738
Train Step # 8739
Train Step # 8740
Train Step # 8741
Train Step # 8742
Train Step # 8743
Train Step # 8744
Train Step # 8745
Train Step # 8746
Train Step # 8747
Train Step # 8748
Train Step # 8749
Train Step # 8750
Train Step # 8751
Train Step # 8752
Train Step # 8753
Train Step # 8754
Train Step # 8755
Train Step # 8756
Train Step # 8757
Train Step # 8758
Train Step

In [ ]:
#save the final output
image_eval = sess.run(image)
best_image_add_mean = image_eval.reshape(shape[1:]) + normalization_mean
output_file = 'final_output.jpg'
scipy.misc.imsave(output_file, best_image_add_mean)